<div align="center">

# Точная настройка моделей с помощью поиска по сетке

</div>

---

**Поиск по сетке (grid search)**, так называемый метод оптимизации гиперпараметров, который способен улучшить производительность модели путем нахождения оптимальной комбинации значений гиперпараметров.

**Принцип работы** - полный перебор варинатов указанных в списках значений гиперпараметров, для поиска оптимальной комбинации максимальной производительности модели.



In [41]:
import numpy as np
import pandas as pd
import scipy.stats 
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [42]:
# Загрузка набора данных из локального хранилища
df = pd.read_csv('~/Рабочий стол/ML/Data/wdbc.data',
                 header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [43]:
# Разделяем данные на
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# Преобразуем метки классов (диагнозы) в целые числа
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

# Проверка сопоставлений le
le.transform(['M', 'B'])

# Разделение на train и test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.20,
                     stratify = y,
                     random_state = 1)


# Создание конвейера SVM
pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state = 1))

param_range = [0.0001, 0.001, 0.01, 0.1,
               1.0, 10.0, 100.0, 1000.0]

# Словарь праметров которые хотим применить к сетке поиска
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator = pipe_svc,
                  param_grid = param_grid,
                  scoring = 'accuracy',
                  cv = 10,
                  refit = True,
                  n_jobs = -1)

gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9846859903381642
{'svc__C': 100.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}


### GridSearchCV для настройки SVM


* Автоматизирует **поиск оптимальных гиперпараметров**.
* Оценивает каждую комбинацию параметров с помощью **k-кратной перекрестной проверки**.


#### **Параметры настройки (param\_grid)**

* **Линейное ядро SVM**:

  * Настраивается только параметр **`C`** — обратный коэффициент регуляризации.

* **RBF-ядро SVM**:

  * **`C`** — обратный коэффициент регуляризации.
  * **`gamma`** — параметр ядра, отвечает за "радиус влияния" опорных векторов.
  * *`gamma` применяется только для ядерных SVM.*


#### **Особенности конфигурации GridSearchCV**

* **`cv=10`** — 10-кратная перекрестная проверка.
* **`scoring='accuracy'`** — оптимизация по средней точности.
* **`n_jobs=-1`** — использование всех ядер процессора (можно сменить на `None` для однопоточной работы).


#### **Результаты работы**

* **`best_score_`** — точность лучшей модели на перекрестной проверке.
* **`best_params_`** — оптимальные параметры.


#### **Финальная проверка**

* После выбора лучшей модели — обязательная **оценка на независимом тестовом наборе**, чтобы проверить реальную обобщающую способность с помощью параметра сетке **`best_estimator_`** — готовая модель с лучшими настройками.

In [44]:
# Оценка на независимом наборе данных
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print(f'Точность при тестировании: {clf.score(X_test, y_test):.3f}')

Точность при тестировании: 0.974


После завершения поиска по сетке нет необходимости вручную обучать модель с найденными оптимальными параметрами (`gs.best_estimator_`) с помощью `clf.fit(X_train, y_train)`.
Если параметр `refit` в `GridSearchCV` установлен в `True` (это значение используется по умолчанию), класс автоматически переобучит лучшую модель (`best_estimator_`) на всём обучающем наборе.

---

## **Randomized Search (рандомизированный поиск)**

* **Суть:** случайным образом выбирает заданное количество комбинаций гиперпараметров из:

  * **Дискретных наборов значений** (как в `GridSearchCV`), или
  * **Распределений** (например, из `scipy.stats`).
* **Преимущества:**

  * Можно охватить **широкий диапазон значений** при меньших вычислительных затратах.
  * Возможность задать распределения (например, логарифмическое) для более равномерного охвата диапазонов с разными масштабами.
* **Контроль:** параметр `n_iter` задаёт количество оцениваемых конфигураций.

#### **Практическая рекомендация**

* **GridSearchCV** — когда гиперпараметров мало и их диапазоны точно определены, но может быть затратен по памяти и времени, а также пропустить хорошие варианты если сетка задана слишком узко. 
* **RandomizedSearchCV** — когда:

  * гиперпараметров много,
  * диапазоны значений большие,
  * нет уверенности в оптимальных пределах поиска.


 **Пример для SVM:**

* Вместо перебора фиксированных значений `C` и `gamma`, можно задать:

  * `C ~ loguniform(0.0001, 100.0)`
  * `gamma ~ loguniform(0.0001, 100.0)`
    Это обеспечит равномерное покрытие **в логарифмическом масштабе** (одинаковое количество выборок в малых и больших диапазонах).

In [45]:
# Распределение для выборки
param_range_1 = scipy.stats.loguniform(0.0001, 1000.0)
np.random.seed(1)
param_range_1.rvs(10)

pipe_svc_1 = make_pipeline(StandardScaler(),
                           SVC(random_state = 1))

param_grid_1 = [{'svc__C': param_range_1,
                 'svc__kernel': ['linear']},
                {'svc__C': param_range_1,
                 'svc__gamma': param_range_1,
                 'svc__kernel': ['rbf']}]

rs = RandomizedSearchCV(estimator = pipe_svc_1,
                        param_distributions = param_grid_1,
                        scoring = 'accuracy',
                        refit = True,
                        n_iter = 20,
                        cv = 10,
                        random_state = 1,
                        n_jobs = -1)

rs = rs.fit(X_train, y_train)
print(rs.best_score_)
print(rs.best_params_)

0.9780676328502416
{'svc__C': np.float64(0.05971247755848463), 'svc__kernel': 'linear'}


`RandomizedSearchCV` очень похож на `GridSearchCV`, за исключением распределения, которое можно задейстовавать в качестве диапозонов параметров и указать кол-во итераций, в нашем случае `n_iter = 20`.

---

## Поиск гиперпараметров методом последовательного деления пополам


**HalvingRandomSearchCV — последовательное деление пополам (scikit-learn)**

**Идея:** более эффективный поиск гиперпараметров за счёт постепенного отбрасывания слабых кандидатов.

**Как работает:**

1. **Генерация кандидатов** — случайная выборка большого набора конфигураций гиперпараметров.
2. **Обучение с ограниченными ресурсами** — например, на небольшом подмножестве данных.
3. **Отбрасывание половины** — исключаем 50% наименее перспективных конфигураций.
4. **Увеличение ресурсов** — оставшиеся модели обучаются на большем объёме данных.

**Повторяем**, пока не останется **1 лучшая конфигурация**.

**Варианты:**

* **HalvingRandomSearchCV** — случайная генерация кандидатов.
* **HalvingGridSearchCV** — полный перебор (grid search) вместо случайных выборок.

**Преимущество:** значительная экономия времени и ресурсов при большом числе конфигураций.

---

In [46]:
# Рандомизированный поиск с последовательным делением пополам
hs = HalvingRandomSearchCV(pipe_svc_1,
                           param_distributions = param_grid_1,
                           n_candidates = 'exhaust',
                           resource = 'n_samples',
                           factor = 1.5,
                           random_state = 1,
                           n_jobs = -1)

hs = hs.fit(X_train, y_train)
print(hs.best_score_)
print(hs.best_params_)
clf_1 = hs.best_estimator_
print(f'Точность при тестировании: {hs.score(X_test, y_test):.3f}')

0.9617647058823529
{'svc__C': np.float64(4.934834261073333), 'svc__kernel': 'linear'}
Точность при тестировании: 0.982


## **HalvingRandomSearchCV — параметры и поведение**

* **`resource='n_samples'`** *(по умолчанию)* — ресурс, который меняется между раундами → размер обучающего набора.
* **`factor`** — доля отсева кандидатов за раунд:

  * `factor=2` → исключаем **50%** кандидатов.
  * `factor=1.5` → в следующий раунд проходит **≈66%** кандидатов.
* **`n_candidates='exhaust'`** *(по умолчанию)* — число конфигураций подбирается так, чтобы в **последнем раунде** использовался **максимум ресурсов**.

**Сравнение эффективности:**

* **HalvingRandomSearchCV** дал точность **98.2%**
* **GridSearchCV** / **RandomizedSearchCV** — около **97.4%**.


**Hyperopt — байесовская оптимизация**

* Популярная библиотека для **поиска гиперпараметров**.
* Поддерживает:

  * Рандомизированный поиск.
  * **TPE** (*Tree-structured Parzen Estimator*) — байесовский метод.
* **TPE**:

  * Строит вероятностную модель зависимости гиперпараметров и качества модели.
  * Модель **постоянно обновляется** на основе предыдущих экспериментов.
  * Оценки **не рассматриваются как независимые** (в отличие от классического случайного поиска).

---

## **Вложенная перекрёстная проверка (Nested Cross-Validation)**

**Когда использовать:**

* Нужно **подбирать гиперпараметры** *и* **сравнивать разные алгоритмы** машинного обучения.
* Минимизация смещения оценки ошибки (по Судхиру Варме и Ричарду Саймону — почти без смещения относительно тестового набора).

**🔹 Как устроена:**

1. **Внешний цикл (k-fold CV)**

   * Делит данные на **обучающую** и **тестовую** части.
2. **Внутренний цикл (k-fold CV)**

   * Работает только на **обучающей части** из внешнего цикла.
   * Подбирает модель и гиперпараметры (GridSearchCV, RandomizedSearchCV и др.).
3. **Оценка**

   * Выбранная модель тестируется на **тестовой части** внешнего цикла.


**Особенности:**

* **Плюс:** почти полное устранение переоценки точности.
* **Минус:** дороже по вычислениям, чем обычная CV.
* **Вариант "5×2"** — 5 раз повторяется схема:

  * Разделение данных на 2 половины (50/50).
  * Обмен обучающей и тестовой частей местами.

In [47]:
param_range_2 = [0.0001, 0.001, 0.01, 0.1,
                 1.0, 10.0, 100.0, 1000.0]
param_grid_2 = [{'svc__C': param_range_2,
                 'svc__kernel': ['linear']},
                {'svc__C': param_range_2,
                 'svc__gamma': param_range_2,
                 'svc__kernel': ['rbf']}]

gs2 = GridSearchCV(estimator = pipe_svc_1,
                   param_grid = param_grid_2,
                   scoring = 'accuracy',
                   cv = 2)

scores = cross_val_score(gs2, X_train, y_train,
                         scoring = 'accuracy', cv = 5)

print(f'Точность перекр. проверки: {np.mean(scores):.3f} '
                      f'+/- {np.std(scores):.3f}')

Точность перекр. проверки: 0.974 +/- 0.015


Средняя точность перекрестной проверки говорит нам о том, чего ожидать, если мы настроим гиперпараметры модели и подадим на ее вход незнакомые данные.

В качестве примера воспользуемся вложенной перекрестной проверкой для сравнения
модели SVM с простым классификатором на основе дерева решений (для наглядности
мы настроим только его параметр глубины).

In [49]:
gs3 = GridSearchCV(
    estimator = DecisionTreeClassifier(random_state = 0),
    param_grid = [{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
    scoring = 'accuracy',
    cv = 2
)

scores_2 = cross_val_score(gs3, X_train, y_train,
                           scoring = 'accuracy', cv = 5)
print(f'Точность перекр. проверки: {np.mean(scores_2):.3f} '
                      f'+/- {np.std(scores_2):.3f}')

Точность перекр. проверки: 0.934 +/- 0.016


В результате **вложенной перекрестной проверки** выяснилось, что производительность **SVM (97.4%)** лучше, чем у **Дерева решений (93.4%)**, поэтому ожидается, что он окажется лучшим выбором классификации новых данных, которые поступают из той же совокупности, что и этот конкретный набор.